# 🧾 View as a summary

In [ ]:
#| default_exp repr_str

In [ ]:
# |hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
#| hide
#| export
from typing import Optional, Union
from collections import defaultdict
from fastcore.foundation import store_attr
import warnings
import numpy as np

from lovely_numpy.utils import pretty_str, PRINT_OPTS, history_warning, sparse_join, np_to_str_common, plain_repr

In [ ]:
# |hide
np.random.seed(0)
randoms = np.random.randn(100)

In [ ]:
nasties = randoms[:12].copy()

nasties[0] *= 10000
nasties[1] /= 10000
nasties[3] = float('inf')
nasties[4] = float('-inf')
nasties[5] = float('nan')
nasties = nasties.reshape((2,6))

In [ ]:
# |exporti
dtnames =   {   "float16": "f16",
                "float32": "f32",
                "float64": "", # Default dtype in numpy
                "uint8": "u8",
                "uint16": "u16",
                "uint32": "u32",
                "uint64": "u64",
                "int8": "i8",
                "int16": "i16",
                "int32": "i32",
                "int64": "i64",
            }

def short_dtype(x: Union[np.ndarray, np.generic]): return dtnames.get(x.dtype.name, str(x.dtype)[6:])

In [ ]:
# |hide
test_eq(short_dtype(np.array(1., dtype=np.float16)), "f16")

In [ ]:
# |exporti

def np_to_str(x: Union[np.ndarray, np.generic],
            plain: bool=False,
            verbose: bool=False,
            depth=0,
            lvl=0,
            color=None) -> str:

    if plain or not np.isrealobj(x):
        return plain_repr(x)
    
    tname = type(x).__name__.split(".")[-1] if isinstance(x, np.ndarray) else None
    shape = str(list(x.shape)) if x.ndim else None
    type_str = sparse_join([tname, shape], sep="")

    color = PRINT_OPTS.color if color is None else color
    common = np_to_str_common(x, color=color)
    dtype = short_dtype(x)
    
    vals = pretty_str(x) if x.size <= 10 else None
    res = sparse_join([type_str, dtype, common, vals])

    if verbose:
        res += "\n" + plain_repr(x)

    if depth and x.ndim > 1:
        res += "\n" + "\n".join([
            " "*PRINT_OPTS.indent*(lvl+1) +
            str(np_to_str(x[i,:], depth=depth-1, lvl=lvl+1))
            for i in range(x.shape[0])])

    return res

In [ ]:
print(np_to_str(nasties))

ndarray[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 +Inf! -Inf! NaN!


In [ ]:
#| exporti

class StrProxy():
    def __init__(self, x: np.ndarray,
                    plain=False,
                    verbose=False,
                    depth=0,
                    lvl=0,
                    color=None):
        store_attr()
        history_warning()
    
    def __repr__(self):
        return np_to_str(self.x, plain=self.plain, verbose=self.verbose,
                      depth=self.depth, lvl=self.lvl, color=self.color)

    # This is used for .deeper attribute and .deeper(depth=...).
    # The second onthe results in a __call__.
    def __call__(self, depth=1):
        return StrProxy(self.x, depth=depth)


Would be _lovely_ if you could see all the important tensor stats too?

In [ ]:
# |export
def lovely(x: np.ndarray, # Tensor of interest
            verbose=False,  # Whether to show the full tensor
            plain=False,    # Just print if exactly as before
            depth=0,        # Show stats in depth
            color=None):    # Force color (True/False) or auto.
    return StrProxy(x, verbose=verbose, plain=plain, depth=depth, color=color)

In [ ]:
print(lovely(randoms[0]))
print(lovely(randoms[:2]))
print(lovely(randoms[:6].reshape(2, 3))) # More than 2 elements -> show statistics
print(lovely(randoms[:11])) # More than 10 -> suppress data output


1.764
ndarray[2] μ=1.082 σ=0.682 [1.764, 0.400]
ndarray[2, 3] n=6 x∈[-0.977, 2.241] μ=1.046 σ=1.090 [[1.764, 0.400, 0.979], [2.241, 1.868, -0.977]]
ndarray[11] x∈[-0.977, 2.241] μ=0.684 σ=0.938


In [ ]:
# |hide
test_eq(str(lovely(randoms[0])), "1.764")
test_eq(str(lovely(randoms[:2])), "ndarray[2] μ=1.082 σ=0.682 [1.764, 0.400]")
test_eq(str(lovely(randoms[:6].reshape(2, 3))), "ndarray[2, 3] n=6 x∈[-0.977, 2.241] μ=1.046 σ=1.090 [[1.764, 0.400, 0.979], [2.241, 1.868, -0.977]]")
test_eq(str(lovely(randoms[:11])), "ndarray[11] x∈[-0.977, 2.241] μ=0.684 σ=0.938")

Do we have __any__ floating point nasties? Is the tensor __all__ zeros?

In [ ]:
# Statistics and range are calculated on good values only, if there are at lest 3 of them.
lovely(nasties)

ndarray[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 +Inf! -Inf! NaN!

In [ ]:
lovely(nasties, color=False)

ndarray[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 +Inf! -Inf! NaN!

In [ ]:
lovely(np.array([float("nan")]*11))

ndarray[11] NaN!

In [ ]:
lovely(np.zeros(12, dtype=np.float16))

ndarray[12] f16 all_zeros

In [ ]:
test_eq(str(lovely(nasties)),
    'ndarray[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 \x1b[31m+Inf!\x1b[0m \x1b[31m-Inf!\x1b[0m \x1b[31mNaN!\x1b[0m')
test_eq(str(lovely(np.array([float("nan")]*11))), 'ndarray[11] \x1b[31mNaN!\x1b[0m')
test_eq(str(lovely(np.zeros(12, dtype=np.float16))), 'ndarray[12] f16 \x1b[38;2;127;127;127mall_zeros\x1b[0m')

In [ ]:
# torch.set_printoptions(linewidth=120)
lovely(nasties, verbose=True)

ndarray[2, 6] n=12 x∈[-0.151, 1.764e+04] μ=1.960e+03 σ=5.544e+03 +Inf! -Inf! NaN!
array([[ 1.76405235e+04,  4.00157208e-05,  9.78737984e-01,
                    inf,            -inf,             nan],
       [ 9.50088418e-01, -1.51357208e-01, -1.03218852e-01,
         4.10598502e-01,  1.44043571e-01,  1.45427351e+00]])

In [ ]:
lovely(nasties, plain=True)

array([[ 1.76405235e+04,  4.00157208e-05,  9.78737984e-01,
                    inf,            -inf,             nan],
       [ 9.50088418e-01, -1.51357208e-01, -1.03218852e-01,
         4.10598502e-01,  1.44043571e-01,  1.45427351e+00]])

In [ ]:
image = np.load("mysteryman.npy")
image[1,100,100] = float('nan')

lovely(image, depth=1)

ndarray[3, 196, 196] f32 n=115248 x∈[-2.118, 2.640] μ=-0.388 σ=1.073 NaN!
  ndarray[196, 196] f32 n=38416 x∈[-2.118, 2.249] μ=-0.324 σ=1.036
  ndarray[196, 196] f32 n=38416 x∈[-1.966, 2.429] μ=-0.274 σ=0.973 NaN!
  ndarray[196, 196] f32 n=38416 x∈[-1.804, 2.640] μ=-0.567 σ=1.178

In [ ]:
# We don't really supposed complex numbers yet
c = np.random.randn(2) + 1j*np.random.randn(2)
lovely(c)

array([ 1.8831507 -1.270485j  , -1.34775906+0.96939671j])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()